In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation, Resizing, Rescaling
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, GaussianNoise
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# from sklearn.metrics import classification_report

In [ ]:
img_width, img_height = 299, 299
train_data_dir = 'Training'
validation_data_dir = 'Testing'
test_data_dir = 'Validation'
nb_train_samples = 7470 
nb_validation_samples = 1511
batch_size = 64
epochs = 30

In [ ]:
training_dataset = tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                            shuffle=True,
                                                            batch_size=batch_size,
                                                            label_mode='categorical',
                                                            image_size=(img_height, img_width),)
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_data_dir,
                                                            shuffle=True,
                                                            label_mode='categorical',
                                                            batch_size=batch_size,
                                                            image_size=(img_height, img_width),)

test_dataset = tf.keras.utils.image_dataset_from_directory(test_data_dir,
                                                            shuffle=True,
                                                            label_mode='categorical',
                                                            batch_size=batch_size,
                                                            image_size=(img_height, img_width),)

training_batches = tf.data.experimental.cardinality(training_dataset)
validation_batches = tf.data.experimental.cardinality(validation_dataset)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = training_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  # tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Resizing(img_height, img_width),
  tf.keras.layers.RandomFlip('horizontal'),
  # tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomContrast(0.2),
  tf.keras.layers.RandomBrightness(0.2),
  # tf.keras.layers.GaussianNoise(0.2),
  
])

In [ ]:
preprocess_input = tf.keras.applications.densenet.preprocess_input

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.DenseNet201(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
# print(feature_batch.shape)

base_model.trainable = False
# base_model.summary()

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
# print(feature_batch_average.shape)

prediction_layer = tf.keras.layers.Dense(7, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
# print(prediction_batch.shape)

inputs = tf.keras.Input(shape=(img_height, img_width, 3))
x = data_augmentation(inputs)
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
# model.summary()

In [ ]:

# Add a global average pooling layer to the base model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Add a fully connected layer with 1024 hidden units and ReLU activation
x = tf.keras.layers.Dense(1024, activation='relu')(x)

# Add a dropout layer to reduce overfitting
x = tf.keras.layers.Dropout(0.5)(x)

# Add a final output layer with 7 units and softmax activation
predictions = tf.keras.layers.Dense(7, activation='softmax')(x)

# Combine the base model and the new output layers
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
nb_classes = 7
adam = Adam(learning_rate=0.1, decay=1e-6)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tfa.metrics.F1Score(num_classes=nb_classes, average='macro')])
# model.summary()


In [ ]:
checkpointer = ModelCheckpoint('model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')
red_plateu = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)

# checkpointer = ModelCheckpoint(filepath='model.hdf5', verbose=1, save_best_only=True, monitor='val_loss')
callbacks = [checkpointer, early_stopping, red_plateu]

In [ ]:
history = model.fit(training_dataset,
          batch_size=batch_size,
        #   steps_per_epoch=training_batches,
          epochs=epochs,
        #   validation_split=0.2,
          validation_data=validation_dataset,
          validation_steps=validation_batches,
          callbacks=callbacks)

In [ ]:
model.save('model_gozu.h5')

In [ ]:
# Función para graficar la historia
import numpy as np
import matplotlib.pyplot as plt
EPOCAS = 17


def graficar(h):
    LOSS = 0
    ACCURACY = 1
    entrenamiento = np.zeros((2, EPOCAS))
    prueba = np.zeros((2, EPOCAS))
    entrenamiento[LOSS] = h.history['loss']  # loss de entrenamiento
    prueba[LOSS] = h.history['val_loss']    # loss de validación
    entrenamiento[ACCURACY] = h.history['accuracy']  # acc de entrenamiento
    prueba[ACCURACY] = h.history['val_accuracy']  # acc de validación
    epocas = range(1, EPOCAS+1)
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    for i, label in zip((LOSS, ACCURACY), ('perdida', 'exactitud')):
        axs[i].plot(epocas, entrenamiento[i], 'b-', label='Entrenamiento '+label)
        axs[i].plot(epocas, prueba[i], 'y-', label='Prueba '+label)
        axs[i].set_title('Entrenamiento y prueba ' + label)
        axs[i].set_xlabel('Epocas')
        axs[i].set_ylabel(label)
        axs[i].legend()
    plt.show()

graficar(history)

### Validación y Resultados

In [ ]:
import numpy
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
X = []
y = []
X_train = []
y_train = []
X_test = []
y_test = []


for images, labels in training_dataset:
    X_train.append(images.numpy())
    y_train.append(labels.numpy())

for images,labels in test_dataset:
    X_test.append(images.numpy())
    y_test.append(labels.numpy())

X = np.concatenate((X_train, X_test), axis=0)

# Necesito crear la variable X, que es la concatenación de X_Train con y_train y lo mismo con X_test y y_test



In [ ]:
X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))


In [ ]:


kfold = StratifiedKFold(n_splits=2,shuffle=True)
#puntuación (accuracy) con cada subconjunto de validación

punt_vc = []
for entr, prue in kfold.split(X,np.zeros(shape=(X_train, 1))):
  modelo = Sequential()
  modelo.add(Dense(256, input_dim=777, activation='relu'))
  modelo.add(Dense(32,activation='relu'))
  modelo.add(Dropout(0.3)) #El 30% de los pesos se hacen igual a cero
  modelo.add(Dense(7,activation='softmax'))
  modelo.compile(loss='categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
  modelo.fit(X_train[entr], y_train[entr], epochs=10, batch_size=64, verbose=0)
  punt = modelo.evaluate(X_test[prue], y_test[prue],verbose=0)
  print("%s: %.2f%%" % (modelo.metrics_names[1],punt[1]*100))
  punt_vc.append(punt[1]*100)

print("%.2f%% (+/-) %.2f%%" % (numpy.mean(punt_vc),numpy.std(punt_vc)))